In [1]:
import Pkg
Pkg.activate("./env")
Pkg.instantiate()

  Activating project at `c:\Users\da93p\Desktop\SCJ\SCJ-Projekt\SCJ-PredatorPrey\env`


In [2]:
# To view our starting population, we can build an overview plot using [`abmplot`](@ref).
# We define the plotting details for the wolves and sheep:
#offset(a) = a.def.type == "Sheep" ? (-0.1, -0.1*rand()) : (+0.1, +0.1*rand())
ashape(a) = a.def.symbol
acolor(a) = a.def.color

# and instruct [`abmplot`](@ref) how to plot grass as a heatmap:
grasscolor(model) = model.growth ./ model.regrowth_time
# and finally define a colormap for the grass:
heatkwargs = (colormap = [:brown, :green], colorrange = (0, 1))

# and put everything together and give it to [`abmplot`](@ref)
return plotkwargs = (;
    agent_color = acolor,
    agent_size = 25,
    agent_marker = ashape,
    #offset,
    agentsplotkwargs = (strokewidth = 1.0, strokecolor = :black),
    heatarray = grasscolor,
    heatkwargs = heatkwargs,
)

(agent_color = acolor, agent_size = 25, agent_marker = ashape, agentsplotkwargs = (strokewidth = 1.0, strokecolor = :black), heatarray = grasscolor, heatkwargs = (colormap = [:brown, :green], colorrange = (0, 1)))

In [48]:
include("./predator_prey_generic.jl")
#Pkg.status(["Agents","GLMakie"]; mode = Pkg.Types.PKGMODE_MANIFEST, io=stdout)
using GLMakie
GLMakie.activate!()

# Scenario 1
#events = [] # -> stabiles System

# Scenario 2
# events = RecurringEvent[
# RecurringEvent("Drought", 30, 40, 100, 124, 240, 0)
# RecurringEvent("Flood", 0.4, 0, 20, 20, 120, 0)
# ] # -> extreme Populationsschwankungen, Räuber stirbt aus (t=433) und System kollabiert

# Scenario 3
# events = RecurringEvent[
# RecurringEvent("Winter", 0, 0, 18, 24, 24, 0)
# RecurringEvent("PreyReproduceSeasonal", 0.5, 0.1, 1, 7, 24, 0)
# RecurringEvent("PredatorReproduceSeasonal", 0.1, 0.04, 6, 12, 24, 0)
# ] # -> geringere max. Population, dafür weniger starke Schwankung.

# Scenario 4
# events = RecurringEvent[
# RecurringEvent("Drought", 30, 40, 100, 124, 240, 0)
# RecurringEvent("Flood", 0.4, 0, 25, 25, 120, 0)
# RecurringEvent("Winter", 0, 0, 18, 24, 24, 0)
# RecurringEvent("PreyReproduceSeasonal", 0.5, 0.1, 1, 7, 24, 0)
# RecurringEvent("PredatorReproduceSeasonal", 0.1, 0.05, 6, 12, 24, 0)
# ] # -> starke Schwankung durch, allerdings bleibt das System stabil

animal_defs = [
AnimalDefinition(30,'●',RGBAf(1.0, 1.0, 1.0, 0.8),20, 0.3, 20, 3, "Sheep", ["Wolf","Bear"], ["Grass"])
AnimalDefinition(3,'▲',RGBAf(0.2, 0.2, 0.3, 0.8),20, 0.07, 20, 1, "Wolf", [], ["Sheep"])
#AnimalDefinition('■',RGBAf(1.0, 0.8, 0.5, 0.8),20, 0.07, 20, 1, "Bear", [], ["Sheep"])
]
parameter_ranges = generate_animal_parameter_ranges(animal_defs)

stable_params = (;
    events = events,
    animal_defs = animal_defs,
    dims = (30, 30),
    regrowth_time = 30,
    Δenergy_grass = 6,
    seed = 71758,
)

params = Dict(
    :regrowth_time => 0:1:100,
    :Δenergy_grass => 0:1:50,
)
params = merge(params,parameter_ranges)
sheep(a) = a.def.type == "Sheep"
wolf(a) = a.def.type == "Wolf"
eaten(a) = a.def.type == "Sheep" && a.death_cause == Predation
starved(a) = a.def.type == "Sheep" && a.death_cause == Starvation
count_grass(model) = count(model.fully_grown)
adata = [(sheep, count), (wolf, count), (eaten, count), (starved, count)]
mdata = [count_grass]
model = initialize_model(;stable_params...)
fig, abmobs = abmexploration(
    model;
    params,
    plotkwargs...,
    adata,
    alabels = ["Sheep", "Wolf", "Eaten", "Starved"],
    mdata, mlabels = ["Grass"]
)
#fig, ax, abmobs = abmplot(model; add_controls=true, plotkwargs...)

fig
#run!(model, 2)